<a href="https://colab.research.google.com/github/SammyDMartin/CC/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
"""
#Uncomment for Colab
!pip install "ray[rllib]" torch
!pip install nashpy
!pip install pickle5
!git clone https://github.com/SammyDMartin/CC
"""

'\n#Uncomment for Colab\n!pip install "ray[rllib]" torch\n!pip install nashpy\n!pip install pickle5\n!git clone https://github.com/SammyDMartin/CC\n'

In [29]:
from cc_search import assess_agent_vs_others

In [30]:
from cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer
from cc_tools import maximin_policy,ideal_selfplay_policy,random_agent
from cc_tools import TFTAverage,RandomPolicy,IteratedPrisonersDilemma

from cc_search import get_rllib_config_eval
import numpy as np
from ray import tune

In [31]:
class LogPolicy(RandomPolicy):
    """Play the move that would beat the last move of the opponent."""
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        self.obs_memory = []
        self.act_memory = []
        self.num = 1

    def update_target(self):
        pass
    def get_weights(self):
        pass
    def compute_actions(self,
                        obs_batch,
                        state_batches=None,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        **kwargs):
        input = obs_batch[0]
        self.obs_memory.append(float(prev_reward_batch))
        self.act_memory.append(float(prev_action_batch))

        output = np.random.choice([0,1])
        n = 12

        #print("Agent{}".format(self.num),state_batches,prev_action_batch,prev_reward_batch)
        #print("Agent{}".format(self.num),input,output,flush=True)

        if len(self.obs_memory)>n+1:
            raise InterruptedError

        if len(self.obs_memory)>n:
            print("Agent{} obs".format(self.num),self.obs_memory)
            print("Agent{} acts".format(self.num),self.act_memory)


        return np.array([output]), \
                [], {}

class LogPolicy2(LogPolicy):
    """Play the move that would beat the last move of the opponent."""
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        self.num = 2


test_pol = lambda x,y: LogPolicy

strategies = [test_pol]
opponents = []

In [32]:
#res1, res_full = assess_agent_vs_others(agent_trainer=(DQNTorchPolicy,DQNTrainer),opponents=opponents, strategies=strategies,steps=2)

rllib_config, stop_config = get_rllib_config_eval(
    seeds=[1],
    steps=3,
    game=IteratedPrisonersDilemma,
    debug=False,
    stop_iters=1,
    policy_classes=[LogPolicy,LogPolicy2],
    checkpoint_paths=[]
)

tune_analysis = tune.run(
    # Works with any Trainer but then using PGTrainer is more safe (simple training loop)... but PGTrainer doesn't work here for some reasons...
    DQNTrainer,
    # PPOTrainer,
    config=rllib_config,
    stop=stop_config,
    checkpoint_at_end=True, verbose = 0
)

 pid=13384) 2022-01-24 14:24:26,807	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=13384) 2022-01-24 14:24:26,825	WARNING util.py:57 -- Install gputil for GPU system monitoring.
 pid=13384) 2022-01-24 14:24:26,834	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!


 pid=13384) Agent1 obs [0.0, -2.0, -1.0, 0.0, -2.0, 0.0, 0.0, -3.0, 0.0, 0.0, -1.0, -3.0, 0.0]
 pid=13384) Agent1 acts [0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
 pid=13384) Agent2 obs [0.0, -2.0, -1.0, 0.0, -2.0, -3.0, 0.0, 0.0, -3.0, 0.0, -1.0, 0.0, 0.0]
 pid=13384) Agent2 acts [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]


2022-01-24 14:24:27,435	ERROR trial_runner.py:958 -- Trial DQN_IteratedPrisonersDilemma_4f81d_00000: Error processing event.
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\ray\tune\trial_runner.py", line 924, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "C:\Users\User\anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 787, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "C:\Users\User\anaconda3\lib\site-packages\ray\_private\client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\User\anaconda3\lib\site-packages\ray\worker.py", line 1713, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(InterruptedError): ray::DQN.train() (pid=13384, ip=127.0.0.1, repr=DQN)
  File "python\ray\_raylet.pyx", line 625, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 629, in ray._raylet.execute_task
  

TuneError: ('Trials did not complete', [DQN_IteratedPrisonersDilemma_4f81d_00000])